# 0.데이터 분석환경 셋팅

## 0.1.라이브러리 불러오기

In [ ]:
#!pip install cufflinks
import os
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display

import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import geopandas as gpd
from shapely.geometry import Polygon, Point 
import shapely # Shapely 형태의 데이터를 받아 내부 좌표들을 List안에 반환합니다. 
import numpy as np
from numpy import random

""" 스케일링 라이브러리 """
from sklearn.preprocessing import MinMaxScaler

""" 시각화 라이브러리 """
import pydeck as pdk
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = 'Nanum Gothic'
import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')

from tqdm.notebook import tqdm

""" 최적화 solver 라이브러리 """
import time
from mip import Model, xsum, maximize, BINARY  

Using Python-MIP package version 1.5.3


## 0.2.데이터 불러오기

In [ ]:
""" 데이터 불러오기 """
df_dodam = gpd.read_file("도담동_50M.geojson")
df_saerom = gpd.read_file("새롬동_50M.geojson")
df_jochi = gpd.read_file("조치원읍_50M.geojson")

df_cafe = pd.read_csv('lonlat_cafe.csv')
df_eat = pd.read_csv('lonlat_eat.csv')
df_house = pd.read_csv('lonlat_house.csv')
df_minwon = pd.read_csv('lonlat_minwon.csv')
df_school = pd.read_csv('lonlat_school.csv')

df_parking = pd.read_csv('lonlat_parking.csv')
df_conven = pd.read_csv('lonlat_convenience.csv')

## 0.3.함수 정의

In [ ]:
""" Scaling Methods에 따른 Scaling하는 함수 """
def col_scaling(data, cols, scale):
    df_cp = data.copy()
    
    if scale == "minmax":
        X = df_cp.loc[:,cols]
        MinMaxScalers = MinMaxScaler()
        MinMaxScalers.fit(X)
        df_MinMaxScaler = MinMaxScalers.transform(X)
        df_cp.loc[:, cols] = df_MinMaxScaler
    elif scale == "standard" :
        X = df_cp.loc[:,cols]
        StandardScalers = StandardScaler()
        StandardScalers.fit(X)
        df_StandardScalers = StandardScalers.transform(X)
        df_cp.loc[:, cols] = df_StandardScalers
    elif scale == "robust" :
        X = df_cp.loc[:,cols]
        RobustScalers = RobustScaler()
        RobustScalers.fit(X)
        df_RobustScaler = RobustScalers.transform(X)
        df_cp.loc [:, cols] = df_RobustScaler
    else:
        print("wrong")
    return df_cp

In [ ]:
""" Pydeck 사용을 위한 함수 정의 """

""" multipolygon를 리스트로 변환하는 함수 """
def multipolygon_to_coordinates(x): 
    lon, lat = x[0].exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

""" polygon를 리스트로 변환하는 함수 """
def polygon_to_coordinates(x): 
    lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

""" 리스트로 변환하는 함수"""
def to_coordinates(x): 
    try: 
        lon, lat = x[0].exterior.xy 
    except:
        lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

""" 격자내 수요지 카운트하는 함수 정의 """
# school의 경우, 반경 300m 존재여부  # 그 외, 반경 100m 존재시 카운트 

def suyo(data,demand,demand_name):
    lon_1m = 1/88.74/1000
    lat_1m = 1/109.96/1000
            
    if demand_name == "df_cafe": 
        data['cafe'] = 0
        for i in range(len(demand)):               
            location = data[(data['lon_max'] <= demand.loc[i,'경도']+lon_1m*100) &
                    (data['lat_max'] <= demand.loc[i,'위도']+lat_1m*100) &
                    (data['lon_min'] >= demand.loc[i,'경도']-lon_1m*100) &
                    (data['lat_min'] >= demand.loc[i,'위도']-lat_1m*100)
                    ].index
            data.loc[location,'cafe'] += 1  
                
    elif demand_name == "df_conven":
        data['convenience'] = 0
        for i in range(len(demand)):               
            location = data[(data['lon_max'] <= demand.loc[i,'경도']+lon_1m*100) &
                    (data['lat_max'] <= demand.loc[i,'위도']+lat_1m*100) &
                    (data['lon_min'] >= demand.loc[i,'경도']-lon_1m*100) &
                    (data['lat_min'] >= demand.loc[i,'위도']-lat_1m*100)
                    ].index
            data.loc[location,'convenience'] += 1
                
    elif demand_name == "df_eat": 
        data['eat'] = 0
        for i in range(len(demand)):               
            location = data[(data['lon_max'] <= demand.loc[i,'경도']+lon_1m*100) &
                    (data['lat_max'] <= demand.loc[i,'위도']+lat_1m*100) &
                    (data['lon_min'] >= demand.loc[i,'경도']-lon_1m*100) &
                    (data['lat_min'] >= demand.loc[i,'위도']-lat_1m*100)
                    ].index
            data.loc[location,'eat'] += 1
                
    elif demand_name == "df_house": 
        data['house'] = 0
        for i in range(len(demand)):               
            location = data[(data['lon_max'] <= demand.loc[i,'경도']+lon_1m*100) &
                    (data['lat_max'] <= demand.loc[i,'위도']+lat_1m*100) &
                    (data['lon_min'] >= -demand.loc[i,'경도']-lon_1m*100) &
                    (data['lat_min'] >= -demand.loc[i,'위도']-lat_1m*100)
                    ].index
            data.loc[location,'house'] += 1
                
    elif demand_name == "df_minwon": 
        data['minwon'] = 0
        for i in range(len(demand)):               
            location = data[(data['lon_max'] <= demand.loc[i,'경도']+lon_1m*100) &
                    (data['lat_max'] <= demand.loc[i,'위도']+lat_1m*100) &
                    (data['lon_min'] >= demand.loc[i,'경도']-lon_1m*100) &
                    (data['lat_min'] >= demand.loc[i,'위도']-lat_1m*100)
                    ].index
            data.loc[location,'minwon'] += 1
                
    elif demand_name == "df_parking": 
        data['parking'] = 0
        for i in range(len(demand)):               
            location = data[(data['lon_max'] <= demand.loc[i,'경도']+lon_1m*100) &
                    (data['lat_max'] <= demand.loc[i,'위도']+lat_1m*100) &
                    (data['lon_min'] >= demand.loc[i,'경도']-lon_1m*100) &
                    (data['lat_min'] >= demand.loc[i,'위도']-lat_1m*100)
                    ].index
            data.loc[location,'parking'] += 1
                
    else:
        demand_name == "df_school" 
        data['school'] = 0
        for i in range(len(demand)):               
            location = data[(data['lon_max'] <= demand.loc[i,'경도']+lon_1m*300) &
                    (data['lat_max'] <= demand.loc[i,'위도']+lat_1m*300) &
                    (data['lon_min'] >= demand.loc[i,'경도']-lon_1m*300) &
                    (data['lat_min'] >= demand.loc[i,'위도']-lat_1m*300)
                    ].index
            data.loc[location,'school'] = 1        

    return data

# 1.데이터 전처리

## 1.1. 데이터 합치기

In [ ]:
""" 행정동 변수 생성 """
df_dodam['dong'] = '도담동'
df_saerom['dong'] = '새롬동'
df_jochi['dong'] = '조치원읍'

In [ ]:
""" 데이터 합치기 """
df_total = pd.concat([df_dodam,df_saerom,df_jochi], ignore_index=True)

""" 필요변수 추출 """
df_total = df_total[['id','50M', 'geometry','dong']] 

df_total.head()  # (2904, 4)

,id,50M,geometry,dong
0,42126.0,1.0,"MULTIPOLYGON (((978245.312 1833080.705, 978345...",도담동
1,42127.0,1.0,"MULTIPOLYGON (((978245.312 1832980.705, 978345...",도담동
2,42124.0,1.0,"MULTIPOLYGON (((978245.312 1833280.705, 978345...",도담동
3,42125.0,1.0,"MULTIPOLYGON (((978245.312 1833180.705, 978345...",도담동
4,42122.0,1.0,"MULTIPOLYGON (((978245.312 1833480.705, 978345...",도담동


## 1.2.격자 필터링

https://www.its.go.kr/nodelink/nodelinkRef

In [ ]:
""" 격자필터링(노드링크 반경 50m이내 격자 추출) """
df_total = df_total[df_total['50M'] == 1].reset_index(drop = True)

df_total.head()  # (1797, 4)

,id,50M,geometry,dong
0,42126.0,1.0,"MULTIPOLYGON (((978245.312 1833080.705, 978345...",도담동
1,42127.0,1.0,"MULTIPOLYGON (((978245.312 1832980.705, 978345...",도담동
2,42124.0,1.0,"MULTIPOLYGON (((978245.312 1833280.705, 978345...",도담동
3,42125.0,1.0,"MULTIPOLYGON (((978245.312 1833180.705, 978345...",도담동
4,42122.0,1.0,"MULTIPOLYGON (((978245.312 1833480.705, 978345...",도담동


## 1.3.좌표계 변환

In [ ]:
""" WGS84(EPSG:4326) 좌표계 변환 : GPS가 사용하는 좌표계(경도와 위도) """
df_total = df_total.to_crs(epsg = 4326)
df_total.head()

,id,50M,geometry,dong
0,42126.0,1.0,"MULTIPOLYGON (((127.25710 36.49513, 127.25821 ...",도담동
1,42127.0,1.0,"MULTIPOLYGON (((127.25710 36.49423, 127.25822 ...",도담동
2,42124.0,1.0,"MULTIPOLYGON (((127.25709 36.49694, 127.25821 ...",도담동
3,42125.0,1.0,"MULTIPOLYGON (((127.25709 36.49604, 127.25821 ...",도담동
4,42122.0,1.0,"MULTIPOLYGON (((127.25709 36.49874, 127.25820 ...",도담동


## 1.4.geometry 관련 변수 생성

In [ ]:
""" coordinates 변수 생성 """
df_total['coordinates'] = df_total['geometry'].apply(multipolygon_to_coordinates) #pydeck 을 위한 coordinate type

In [ ]:
""" lat_min, lat_max, lon_min, lon_max 변수 생성 """
lat_min = []
lat_max = []
lon_min = []
lon_max = []
for i in range(len(df_total)):  
    lon_min.append(df_total['coordinates'][i][0][0])
    lon_max.append(df_total['coordinates'][i][1][0])
    lat_min.append(df_total['coordinates'][i][2][1])
    lat_max.append(df_total['coordinates'][i][0][1])
df_total['lon_min'] = lon_min
df_total['lon_max'] = lon_max
df_total['lat_min'] = lat_min
df_total['lat_max'] = lat_max

In [ ]:
""" coord_cent, geo_cent 변수 생성 """
df_total_list = []
df_total_list2 = []
for i in df_total['geometry']:
    cent = [[i[0].centroid.coords[0][0],i[0].centroid.coords[0][1]]]
    df_total_list.append(cent)
    df_total_list2.append(Point(cent[0]))
df_total['coord_cent'] = 0
df_total['geo_cent'] = 0
df_total['coord_cent']= pd.DataFrame(df_total_list)  # pydeck을 위한 coordinate type
df_total['geo_cent'] = df_total_list2  # geopandas를 위한 geometry type

df_total.head()

,id,50M,geometry,dong,coordinates,lon_min,lon_max,lat_min,lat_max,coord_cent,geo_cent
0,42126.0,1.0,"MULTIPOLYGON (((127.25710 36.49513, 127.25821 ...",도담동,"[[127.25709627500409, 36.495133743822834], [12...",127.257096,127.258213,36.494235,36.495134,"[127.25765595394668, 36.49468412283486]",POINT (127.2576559539467 36.49468412283486)
1,42127.0,1.0,"MULTIPOLYGON (((127.25710 36.49423, 127.25822 ...",도담동,"[[127.2570990902741, 36.49423223224252], [127....",127.257099,127.258216,36.493333,36.494232,"[127.257658762684, 36.493782611149065]",POINT (127.257658762684 36.49378261114907)
2,42124.0,1.0,"MULTIPOLYGON (((127.25709 36.49694, 127.25821 ...",도담동,"[[127.25709064408868, 36.49693676657332], [127...",127.257091,127.258207,36.496038,36.496937,"[127.25765033609753, 36.49648714579628]",POINT (127.2576503360975 36.49648714579628)
3,42125.0,1.0,"MULTIPOLYGON (((127.25709 36.49604, 127.25821 ...",도담동,"[[127.25709345960895, 36.496035255266435], [12...",127.257093,127.258210,36.495136,36.496035,"[127.25765314508455, 36.49558563438393]",POINT (127.2576531450845 36.49558563438393)
4,42122.0,1.0,"MULTIPOLYGON (((127.25709 36.49874, 127.25820 ...",도담동,"[[127.25708501267265, 36.49873978877693], [127...",127.257085,127.258202,36.497841,36.498740,"[127.25764471774896, 36.498290168210865]",POINT (127.257644717749 36.49829016821086)


## 1.5.수요지 관련 카운트 변수 생성

In [ ]:
df_total= suyo(df_total,df_cafe,"df_cafe")
df_total= suyo(df_total,df_conven,"df_conven")
df_total= suyo(df_total,df_eat,"df_eat")
df_total= suyo(df_total,df_house,"df_house")
df_total= suyo(df_total,df_minwon,"df_minwon")
df_total= suyo(df_total,df_parking,"df_parking")
df_total= suyo(df_total,df_school,"df_school")

In [ ]:
df_total.describe()

,id,50M,lon_min,lon_max,lat_min,lat_max,cafe,convenience,eat,house,minwon,parking,school
count,1797.000000,1797.0,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000
mean,47256.021146,1.0,127.272867,127.273984,36.546316,36.547215,0.208125,0.076238,1.017251,134.105175,0.058431,0.086811,0.299388
std,5521.110629,0.0,0.016970,0.016971,0.051413,0.051413,0.809933,0.302693,3.759507,85.036233,0.308442,0.379351,0.458117
min,35892.000000,1.0,127.237975,127.239092,36.477080,36.477979,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,42852.000000,1.0,127.259324,127.260440,36.499670,36.500569,0.000000,0.000000,0.000000,42.000000,0.000000,0.000000,0.000000
50%,45732.000000,1.0,127.268170,127.269287,36.526700,36.527599,0.000000,0.000000,0.000000,180.000000,0.000000,0.000000,0.000000
75%,52563.000000,1.0,127.289185,127.290303,36.600683,36.601583,0.000000,0.000000,0.000000,199.000000,0.000000,0.000000,1.000000
max,58026.000000,1.0,127.305963,127.307081,36.636735,36.637635,10.000000,3.000000,57.000000,260.000000,4.000000,6.000000,1.000000


## 1.6.카운트 변수 스케일링

In [ ]:
# " 데이터프레임으로 변환(pydeck은 pandas.dataframe만 인식) """
# df_scaler = pd.DataFrame(df_scaler).reset_index(drop = True)

df_scaler = df_total.copy()

In [ ]:
""" 카운트 변수 minmax 스케일링 """
df_scaler['cafe'] = col_scaling(df_scaler, cols=['cafe'], scale='minmax')['cafe']
df_scaler['convenience'] = col_scaling(df_scaler, cols=['convenience'], scale='minmax')['convenience']
df_scaler['eat'] = col_scaling(df_scaler, cols=['eat'], scale='minmax')['eat']
df_scaler['house'] = col_scaling(df_scaler, cols=['house'], scale='minmax')['house']
df_scaler['minwon'] = col_scaling(df_scaler, cols=['minwon'], scale='minmax')['minwon']
df_scaler['parking'] = col_scaling(df_scaler, cols=['parking'], scale='minmax')['parking']
df_scaler['school'] = col_scaling(df_scaler, cols=['school'], scale='minmax')['school']

In [ ]:
df_scaler['val1'] = df_scaler['cafe']+df_scaler['eat']+df_scaler['house']+df_scaler['minwon']+df_scaler['school']
df_scaler['val2'] = df_scaler['convenience']+df_scaler['parking']
df_scaler['tot_val'] = df_scaler['cafe']+df_scaler['convenience']+df_scaler['eat']+df_scaler['house']+df_scaler['minwon']+df_scaler['parking']+df_scaler['school']

In [ ]:
df_scaler.describe()

,id,50M,lon_min,lon_max,lat_min,lat_max,cafe,convenience,eat,house,minwon,parking,school,val1,val2,tot_val
count,1797.000000,1797.0,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000
mean,47256.021146,1.0,127.272867,127.273984,36.546316,36.547215,0.020812,0.025413,0.017847,0.515789,0.014608,0.014469,0.299388,0.868444,0.039881,0.908325
std,5521.110629,0.0,0.016970,0.016971,0.051413,0.051413,0.080993,0.100898,0.065956,0.327062,0.077110,0.063225,0.458117,0.621049,0.126161,0.644919
min,35892.000000,1.0,127.237975,127.239092,36.477080,36.477979,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,42852.000000,1.0,127.259324,127.260440,36.499670,36.500569,0.000000,0.000000,0.000000,0.161538,0.000000,0.000000,0.000000,0.361538,0.000000,0.411538
50%,45732.000000,1.0,127.268170,127.269287,36.526700,36.527599,0.000000,0.000000,0.000000,0.692308,0.000000,0.000000,0.000000,0.734615,0.000000,0.757692
75%,52563.000000,1.0,127.289185,127.290303,36.600683,36.601583,0.000000,0.000000,0.000000,0.765385,0.000000,0.000000,1.000000,1.248313,0.000000,1.337314
max,58026.000000,1.0,127.305963,127.307081,36.636735,36.637635,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.180364,1.166667,3.680364


## 1.7.후보지 필터링

In [ ]:
""" 후보지필터링(수요가 없는 후보지 제외) """
df = df_scaler[df_scaler['tot_val'] != 0]

## 1.8.시각화

In [ ]:
# """ mapbox api token """
# MAPBOX_API_KEY = "pk.eyJ1IjoiZG9yYWtpbSIsImEiOiJja3R4bmwyb3ExNWdvMndxbWd5ZzZhYmw1In0.K5CTJ8_8pLhdwxKLrkK4hQ"

In [ ]:
# """ icon layer 생성 """
# icon_data = {
#     # Icon from Wikimedia, used the Creative Commons Attribution-Share Alike 3.0
#     # Unported, 2.5 Generic, 2.0 Generic and 1.0 Generic licenses
#     "url": "https://upload.wikimedia.org/wikipedia/commons/1/11/%EC%9C%84%EC%B9%98_%ED%91%9C%EC%8B%9C.png",
#     "width": 242,
#     "height": 242,
#     "anchorY": 242,
# }

# df_school["icon_data"] = None
# for i in df_school.index:
#     df_school["icon_data"][i] = icon_data
    
# school_sign = pdk.Layer(
#     type="IconLayer",
#     data=df_school,
#     get_icon="icon_data",
#     get_size=2,
#     size_scale=7,
#     get_position=["경도", "위도"],
#     pickable=True,
# )

In [ ]:
# """ grid layer 생성"""

# layer0 = pdk.Layer(
#     'PolygonLayer', # 사용할 Layer 타입
#     df_scaler, # 시각화에 쓰일 데이터프레임
#     get_polygon = 'coordinates', # geometry 정보를 담고있는 컬럼 이름
#     get_fill_color = '[50,50, 50, 100]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
#     pickable = True, # 지도와 interactive 한 동작 on
#     auto_highlight = True # 마우스 오버(hover) 시 박스 출력
# )

# layer = pdk.Layer(
#     'PolygonLayer', # 사용할 Layer 타입
#     df, # 시각화에 쓰일 데이터프레임
#     get_polygon = 'coordinates', # geometry 정보를 담고있는 컬럼 이름
#     get_fill_color = '[50, 150, 50, 255]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
#     pickable = True, # 지도와 interactive 한 동작 on
#     auto_highlight = True # 마우스 오버(hover) 시 박스 출력
# )

# school_layer = pdk.Layer(
#     'PolygonLayer', # 사용할 Layer 타입
#     df, # 시각화에 쓰일 데이터프레임
#     get_polygon = 'coordinates', # geometry 정보를 담고있는 컬럼 이름
#     get_fill_color = '[100*school, 50, 50, 255]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
#     pickable = True, # 지도와 interactive 한 동작 on
#     auto_highlight = True # 마우스 오버(hover) 시 박스 출력
# )

# val1_layer = pdk.Layer(
#     'PolygonLayer', # 사용할 Layer 타입
#     df, # 시각화에 쓰일 데이터프레임
#     get_polygon = 'coordinates', # geometry 정보를 담고있는 컬럼 이름
#     get_fill_color = '[100*val1, 50, 50, 255]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
#     pickable = True, # 지도와 interactive 한 동작 on
#     auto_highlight = True # 마우스 오버(hover) 시 박스 출력
# )

# val2_layer = pdk.Layer(
#     'PolygonLayer', # 사용할 Layer 타입
#     df, # 시각화에 쓰일 데이터프레임
#     get_polygon = 'coordinates', # geometry 정보를 담고있는 컬럼 이름
#     get_fill_color = '[200*val2, 50, 50, 255]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
#     pickable = True, # 지도와 interactive 한 동작 on
#     auto_highlight = True # 마우스 오버(hover) 시 박스 출력
# )

# totval_layer = pdk.Layer(
#     'PolygonLayer', # 사용할 Layer 타입
#     df, # 시각화에 쓰일 데이터프레임
#     get_polygon = 'coordinates', # geometry 정보를 담고있는 컬럼 이름
#     get_fill_color = '[100*tot_val, 50, 50, 255]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
#     pickable = True, # 지도와 interactive 한 동작 on
#     auto_highlight = True # 마우스 오버(hover) 시 박스 출력
# )

In [ ]:
# """ viewpoint 지정 """
# center = [127.271533, 36.592541]
# view_state = pdk.ViewState(
#     longitude = center[0],
#     latitude = center[1],
#     zoom = 10)

In [ ]:
# """ render """
# r = pdk.Deck(
#              layers = [layer0,layer], 
#              initial_view_state = view_state,
#              map_style='mapbox://styles/mapbox/outdoors-v11',
#              mapbox_key = MAPBOX_API_KEY)
# # r.show()
# r.to_html("layer.html")

In [ ]:
# """ render """
# r = pdk.Deck(
#              layers = [school_layer, school_sign], 
#              initial_view_state = view_state,
#              map_style='mapbox://styles/mapbox/outdoors-v11',
#              mapbox_key = MAPBOX_API_KEY)
# # r.show()
# r.to_html("school_layer.html")

In [ ]:
# r = pdk.Deck(
#              layers = [val1_layer], 
#              initial_view_state = view_state,
#              map_style='mapbox://styles/mapbox/outdoors-v11',
#              mapbox_key = MAPBOX_API_KEY)
# # r.show()
# r.to_html("val1_layer.html")

In [ ]:
# r = pdk.Deck(
#              layers = [val2_layer], 
#              initial_view_state = view_state,
#              map_style='mapbox://styles/mapbox/outdoors-v11',
#              mapbox_key = MAPBOX_API_KEY)
# # r.show()
# r.to_html("val2_layer.html")

In [ ]:
# r = pdk.Deck(
#              layers = [totval_layer], 
#              initial_view_state = view_state,
#              map_style='mapbox://styles/mapbox/outdoors-v11',
#              mapbox_key = MAPBOX_API_KEY)
# # r.show()
# r.to_html("totval_layer_layer.html")

## 1.9.행정동별 데이터 분리

In [ ]:
""" 필요변수 추출 """
df = df[['id','geometry','dong','tot_val']] 
df.head()  # (1725, 4)

,id,geometry,dong,tot_val
0,42126.0,"MULTIPOLYGON (((127.25710 36.49513, 127.25821 ...",도담동,0.834615
1,42127.0,"MULTIPOLYGON (((127.25710 36.49423, 127.25822 ...",도담동,0.838462
2,42124.0,"MULTIPOLYGON (((127.25709 36.49694, 127.25821 ...",도담동,0.830769
3,42125.0,"MULTIPOLYGON (((127.25709 36.49604, 127.25821 ...",도담동,0.983401
4,42122.0,"MULTIPOLYGON (((127.25709 36.49874, 127.25820 ...",도담동,0.826923


In [ ]:
""" 행정동별 데이터 추출 """
df_dodam = df[df['dong'] == '도담동'].reset_index(drop = 'True')
df_saerom= df[df['dong'] == '새롬동'].reset_index(drop = 'True')
df_jochi = df[df['dong'] == '조치원읍'].reset_index(drop = 'True')

In [ ]:
""" 데이터 내보내기 """
# df_dodam.shape  # (734, 4)
df_dodam.to_file('df_dodam.geojson', driver='GeoJSON')

In [ ]:
# df_saerom.shape  # (317, 4)
df_saerom.to_file('df_saerom.geojson', driver='GeoJSON')

In [ ]:
# df_jochi.shape # (674, 4)
df_jochi.to_file('df_jochi.geojson', driver='GeoJSON')